# Détection de dérive dans les données de diabète

![concept-drift-detection-methods](assets/8-concept-drift-detection-methods.png.webp)

En suivant ce TP, vous serez capable de détecter la dérive dans les données de diabète et de prendre les mesures appropriées pour maintenir la performance du modèle.

| Concept   | Définition                                                                                                                         | Exemple                                                                                                                          |
|-----------|-------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------|
| Accuracy  | La proportion de diagnostics corrects parmi tous les diagnostics effectués.                                                        | Si tu diagnostiques correctement 90 patients sur 100, ton accuracy est de 90%.                                                   |
| Precision | La proportion de vrais positifs (diagnostics corrects de la maladie) parmi tous les diagnostics positifs (diagnostics de la maladie). | Si tu diagnostiques 30 patients comme malades et 25 d'entre eux le sont vraiment, ta précision est de 83.3%.                    |
| Recall    | La proportion de vrais positifs parmi tous les vrais cas (patients réellement malades).                                               | Si 40 patients sont réellement malades et tu diagnostiques correctement 25 d'entre eux, ton rappel est de 62.5%.                  |
| F1 Score  | La moyenne harmonique de la précision et du rappel.                                                                                  | Si ta précision est de 83.3% et ton rappel est de 62.5%, ton F1 score est de 71.4%.                                             |


In [1]:
#Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from scipy.stats import ks_2samp
import itertools
import time
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np


Importons les bibliothèques nécessaires pour manipuler les données, créer et évaluer un modèle de régression logistique, détecter la dérive des données, et visualiser les résultats.

In [2]:
# Charger le dataset de diabète
url = 'diabetes.csv'
df = pd.read_csv(url)

# Définir les colonnes
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
target_column = 'Outcome'


Chargement du jeu de données et définissons les colonnes des caractéristiques et la colonne cible.



In [3]:
#Fonction pour introduire la dérive dans les données
def introduce_drift(data, drift_probability=1):
    drifted_data = data.copy()
    if np.random.rand() < drift_probability:
        original_glucose = drifted_data['Glucose'].values[0]
        original_bmi = drifted_data['BMI'].values[0]
        drifted_data['Glucose'] *= np.random.uniform(3.0, 5.0) 
        drifted_data['BMI'] *= np.random.uniform(0.2, 0.5)
        print(f"Original Glucose: {original_glucose}, Drifted Glucose: {drifted_data['Glucose'].values[0]}")
        print(f"Original BMI: {original_bmi}, Drifted BMI: {drifted_data['BMI'].values[0]}")
    return drifted_data


Ici on cherche à introduit une dérive dans les valeurs de glucose et d'IMC des données.

In [4]:
#Ingestion des données avec introduction de dérive
def ingest_data_stream_with_drift(df, drift_probability=1):
    # Créer un itérateur infini sur le dataset
    data_iterator = itertools.cycle(df.to_dict('records'))
    while True:
        data = next(data_iterator)
        # Introduire la dérive avec une certaine probabilité
        data_with_drift = introduce_drift(pd.DataFrame([data]), drift_probability)
        yield data_with_drift.to_dict('records')[0]
        time.sleep(2)  # Introduire un délai de 2 secondes


Cette fonction simule un flux de données en introduisant de la dérive à chaque point de données avec une certaine probabilité.

In [5]:
#Détection de dérive
def detect_drift(X_train, X_test):
    # Effectuer le test de Kolmogorov-Smirnov pour détecter la dérive des données
    drift_scores = []
    for col in X_train.columns:
        drift_score = ks_2samp(X_train[col], X_test[col]).statistic
        drift_scores.append(drift_score)
        print(f"Score drift pour {col}: {drift_score:.2f}")
    
    # Vérifier si le score de dérive maximal dépasse un seuil
    drift_threshold = 0.09
    if max(drift_scores) > drift_threshold:
        return True, drift_scores
    else:
        return False, drift_scores


Cette fonction utilise le test de Kolmogorov-Smirnov pour détecter la dérive entre les données d'entraînement et de test.

In [6]:
#Notification et recommandations
def notify_and_recommend(drift_detected, drift_scores, columns, model, X_test, y_test):
    if drift_detected:
        print("Dérive des données détectée !")
        print("Actions recommandées :")
        print("- Réentraîner le modèle avec les données les plus récentes")
        print("- Étudier les caractéristiques ayant des scores de dérive élevés :")
        for i, score in enumerate(drift_scores):
            print(f"  - {columns[i]}: {score:.2f}")
        print("- Collecter des données supplémentaires pour améliorer les performances du modèle")
    else:
        print("Aucune dérive des données n'a été détectée. (Model is up-to-date)")

    # Calculer les métriques de performance après réentraînement
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Afficher les nouvelles métriques de performance
    print(f"\nPerformance du modèle au {datetime.now()}:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

    return accuracy, precision, recall, f1


- Cette fonction vas notifier l'utilisateur de la détection de dérive et recommande des actions. 
- Elle calcule également les métriques de performance du modèle.

In [7]:
#Suivi de la distribution des diabétiques
def track_diabetic_distribution(df):
    df['Timestamp'] = pd.to_datetime(df.index, unit='s')
    df['Diabetic'] = df['Outcome']
    df.set_index('Timestamp', inplace=True)
    df['Diabetic'].resample('D').mean().plot(title='Répartition des diabétiques au fil du temps')
    plt.xlabel('Date')
    plt.ylabel('Proportion de diabétiques')
    plt.show()


In [8]:
# Entrainement initiale
X = df[feature_columns]
y = df[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Mesures de performance initiales
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

accuracy_history = [accuracy]
precision_history = [precision]
recall_history = [recall]
f1_history = [f1]

# DataFrame to store drifted data
drifted_data_list = []

print("\nPerformance initiale du modèle:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")



Performance initiale du modèle:
Accuracy: 0.75
Precision: 0.64
Recall: 0.67
F1 Score: 0.65


Effectuons un entraînement initial du modèle et calculons les métriques de performance initiales.

In [13]:
# Suivi de la distribution des diabétiques
#track_diabetic_distribution(df)


La boucle principale qui ingère les nouvelles données, détecte la dérive, réentraîne le modèle et notifie l'utilisateur tout en enregistrant les données dérivées et les métriques de performance.


In [ ]:
# Boucle principale pour ingérer de nouvelles données, détecter la dérive et réentraîner le modèle

data_stream = ingest_data_stream_with_drift(df, drift_probability=1)

for i in range(30):  # generation de 30 points de données pour test
    new_data = next(data_stream)
    X_new = pd.DataFrame([new_data])[feature_columns]
    y_new = pd.DataFrame([new_data])[target_column]

    # Nettoyez les valeurs manquantes
    X_new = X_new.dropna()
    y_new = y_new.dropna()
    
    if not X_new.empty and not y_new.empty:
        # Ajoutez les nouvelles données
        X_train = pd.concat([X_train, X_new], ignore_index=True)
        y_train = pd.concat([y_train, y_new], ignore_index=True)

        # Réentraînez le modèle
        model.fit(X_train, y_train)

        # Détection de dérive
        drift_detected, drift_scores = detect_drift(X_train, X_test)

        # Notification et recommandations
        accuracy, precision, recall, f1 = notify_and_recommend(drift_detected, drift_scores, feature_columns, model, X_test, y_test)

        # Enregistrez les données avec dérive
        if drift_detected:
            drifted_data_list.append(new_data)

        # Calcul des métriques de performance
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Enregistrez les métriques de performance
        accuracy_history.append(accuracy)
        precision_history.append(precision)
        recall_history.append(recall)
        f1_history.append(f1)

        # Save the drifted data and performance metrics to CSV files
        pd.DataFrame(drifted_data_list).to_csv('drifted_data.csv', index=False)
        metrics_df = pd.DataFrame({
            'datetime': datetime.now(),
            'accuracy': accuracy_history,
            'precision': precision_history,
            'recall': recall_history,
            'f1_score': f1_history
        })
        metrics_df.to_csv('performance_metrics.csv', index=False)
